<a href="https://colab.research.google.com/github/ZitongZhou/react_inverse/blob/master/ILUES_surrogate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install attrdict
!pip install --upgrade matplotlib

Requirement already up-to-date: matplotlib in /usr/local/lib/python3.7/dist-packages (3.4.1)


In [7]:
from tqdm import tqdm
import numpy as np
import pickle as pk
import time
import sys
import os
import copy
import torch
import scipy.io
from attrdict import AttrDict
import argparse
from google.colab import drive
from google.colab import files
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as utils
from torch.utils.data import sampler
from torch.optim.lr_scheduler import ReduceLROnPlateau, StepLR, MultiStepLR
from torch.autograd import Variable
from torch.nn.parameter import Parameter
import torchvision.datasets as dset
import torchvision.transforms as T
import torch.nn.functional as F
from torch.distributions import Gamma
from torchsummary import summary
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib.colors
import matplotlib.backends.backend_pdf
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# DenseED, Bayesian DenseED, and CAAE

In [60]:
"""
Convolutional Dense Encoder-Decoder Networks

Reference:
    https://github.com/pytorch/vision/blob/master/torchvision/models/densenet.py

Yinhao Zhu
Dec 21, 2017
Dec 30, 2017
Jan 03, 2018

Shaoxing Mo
May 07, 2019

Zitong Zhou
Feb 18, 2021
"""


class _DenseLayer(nn.Sequential):
    # bottleneck layer, bn_size: bottleneck size
    def __init__(self, in_features, growth_rate, drop_rate=0, bn_size=4,
                 bottleneck=False):
        # detect if the input features are more than bn_size x k,
        # if yes, use bottleneck -- not much memory gain, but lose one relu
        # I disabled the bottleneck for current implementation
        super(_DenseLayer, self).__init__()
        if bottleneck and in_features > bn_size * growth_rate:
            self.add_module('norm1', nn.BatchNorm3d(in_features))
            self.add_module('relu1', nn.ReLU(inplace=True))
            self.add_module('conv1', nn.Conv3d(in_features, bn_size *
                            growth_rate, kernel_size=1, stride=1, bias=False))
            self.add_module('norm2', nn.BatchNorm3d(bn_size * growth_rate))
            self.add_module('relu2', nn.ReLU(inplace=True))
            self.add_module('conv2', nn.Conv3d(bn_size * growth_rate, growth_rate,
                            kernel_size=3, stride=1, padding=1, bias=False))
        else:
            self.add_module('norm1', nn.BatchNorm3d(in_features))
            self.add_module('relu1', nn.ReLU(inplace=True))
            self.add_module('conv1', nn.Conv3d(in_features, growth_rate,
                            kernel_size=3, stride=1, padding=1, bias=False))
        self.drop_rate = drop_rate

    def forward(self, x):
        y = super(_DenseLayer, self).forward(x)
        if self.drop_rate > 0:
            y = F.dropout3d(y, p=self.drop_rate, training=self.training)
        z = torch.cat([x, y], 1)
        return z


class _DenseBlock(nn.Sequential):
    def __init__(self, num_layers, in_features, growth_rate, drop_rate,
                 bn_size=4, bottleneck=False):
        super(_DenseBlock, self).__init__()
        for i in range(num_layers):
            layer = _DenseLayer(in_features + i * growth_rate, growth_rate,
                                drop_rate=drop_rate, bn_size=bn_size,
                                bottleneck=bottleneck)
            self.add_module('denselayer%d' % (i + 1), layer)

class _Transition(nn.Sequential):
    def __init__(self, in_features, out_features, encoding=True, drop_rate=0.,
                 last=False, out_channels=3, outsize_even=True):
        super(_Transition, self).__init__()
        self.add_module('norm1', nn.BatchNorm3d(in_features))
        self.add_module('relu1', nn.ReLU(inplace=True))
        if encoding:
            # reduce feature maps; half image size (input feature size is even)
            # bottleneck impl, save memory, add nonlinearity
            self.add_module('conv1', nn.Conv3d(in_features, out_features,
                                              kernel_size=1, stride=1,
                                              padding=0, bias=False))
            if drop_rate > 0:
                self.add_module('dropout1', nn.Dropout3d(p=drop_rate))
            self.add_module('norm2', nn.BatchNorm3d(out_features))
            self.add_module('relu2', nn.ReLU(inplace=True))
            self.add_module('conv2', nn.Conv3d(out_features, out_features,
                                              kernel_size=3, stride=2,
                                              padding=1, bias=False))
            if drop_rate > 0:
                self.add_module('dropout2', nn.Dropout3d(p=drop_rate))
        else:
            # decoding, transition up
            if last:
                ks = 6 if outsize_even else 5
                out_convt = nn.ConvTranspose3d(out_features, out_channels,
                                kernel_size=[2,ks,ks], stride=2, padding=[0,2,2],
                                output_padding=[0,0,0], bias=False)
            else:
                out_convt = nn.ConvTranspose3d(
                    out_features, out_features, kernel_size=3, stride=2,
                    padding=1, output_padding=0, bias=False)

            # bottleneck impl, save memory, add nonlinearity
            self.add_module('conv1', nn.Conv3d(in_features, out_features,
                                              kernel_size=1, stride=1,
                                              padding=0, bias=False))
            if drop_rate > 0:
                self.add_module('dropout1', nn.Dropout3d(p=drop_rate))

            self.add_module('norm2', nn.BatchNorm3d(out_features))
            self.add_module('relu2', nn.ReLU(inplace=True))
            self.add_module('convT2', out_convt)
            if drop_rate > 0:
                self.add_module('dropout2', nn.Dropout3d(p=drop_rate))

class DenseED(nn.Module):
    def __init__(self, in_channels, out_channels, blocks, growth_rate=16,
                 num_init_features=64, bn_size=4, drop_rate=0, outsize_even=False,
                 bottleneck=False):
        """
        Args:
            in_channels (int): number of input channels
            out_channels (int): number of output channels
            blocks: list (of odd size) of integers
            growth_rate (int): K
            num_init_features (int): the number of feature maps after the first
                conv layer
            bn_size: bottleneck size for number of feature maps (not useful...)
            bottleneck (bool): use bottleneck for dense block or not
            drop_rate (float): dropout rate
            outsize_even (bool): if the output size is even or odd (e.g.
                65 x 65 is odd, 64 x 64 is even)

        """
        super(DenseED, self).__init__()
        self.out_channels = out_channels

        if len(blocks) > 1 and len(blocks) % 2 == 0:
            ValueError('length of blocks must be an odd number, but got {}'
                       .format(len(blocks)))
        enc_block_layers = blocks[: len(blocks) // 2]
        dec_block_layers = blocks[len(blocks) // 2:]
        self.features = nn.Sequential()
        # First convolution ================
        # only conv, half image size
        self.features.add_module('in_conv',
                    nn.Conv3d(in_channels, num_init_features,
                            kernel_size=[3,7,7], stride=2, padding=[1,3,3], bias=False))

        # Encoding / transition down ================
        # dense block --> encoding --> dense block --> encoding
        num_features = num_init_features
        for i, num_layers in enumerate(enc_block_layers):
            block = _DenseBlock(num_layers=num_layers,
                                in_features=num_features,
                                bn_size=bn_size, growth_rate=growth_rate,
                                drop_rate=drop_rate, bottleneck=bottleneck)
            self.features.add_module('encblock%d' % (i + 1), block)
            num_features = num_features + num_layers * growth_rate

            trans = _Transition(in_features=num_features,
                                out_features=num_features // 2,
                                encoding=True, drop_rate=drop_rate)
            self.features.add_module('down%d' % (i + 1), trans)
            num_features = num_features // 2

        # Decoding / transition up ==============
        # dense block --> decoding --> dense block --> decoding --> dense block
        # if len(dec_block_layers) - len(enc_block_layers) == 1:
        for i, num_layers in enumerate(dec_block_layers):
            block = _DenseBlock(num_layers=num_layers,
                                in_features=num_features,
                                bn_size=bn_size, growth_rate=growth_rate,
                                drop_rate=drop_rate, bottleneck=bottleneck)
            self.features.add_module('decblock%d' % (i + 1), block)
            num_features += num_layers * growth_rate

            # if this is the last decoding layer is the output layer
            last_layer = True if i == len(dec_block_layers) - 1 else False

            trans = _Transition(in_features=num_features,
                                out_features=num_features // 2,
                                encoding=False, drop_rate=drop_rate,
                                last=last_layer, out_channels=out_channels,
                                outsize_even=outsize_even)
            self.features.add_module('up%d' % (i + 1), trans)
            num_features = num_features // 2

    def forward(self, x):
        y = self.features(x)

        #set the source point conc to be constant if there's release
        nonzero_source = torch.nonzero(x[:,2,:,:,:])
        # print('nonzero ',nonzero_source)
        if len(nonzero_source) > 0:
            for non in nonzero_source:
                y[non[0],0,non[1],non[2],non[3]] = x[non[0],2,non[1],non[2],non[3]]
                
        # use the softplus activation for concentration and head
        y = F.softplus(y.clone(), beta=5)

        # CAUTION: if the last channel is pressure, 
        # use the sigmoid activation for pressure
        # y[:,self.out_channels-1] = torch.sigmoid(y[:,self.out_channels-1])

        return y

    def _num_parameters_convlayers(self):
        n_params, n_conv_layers = 0, 0
        for name, param in self.named_parameters():
            if 'conv' in name:
                n_conv_layers += 1
            n_params += param.numel()
        return n_params, n_conv_layers

    def _count_parameters(self):
        n_params = 0
        for name, param in self.named_parameters():
            print(name)
            print(param.size())
            print(param.numel())
            n_params += param.numel()
            print('num of parameters so far: {}'.format(n_params))

    def reset_parameters(self, verbose=False):
        for module in self.modules():
            # pass self, otherwise infinite loop
            if isinstance(module, self.__class__):
                continue
            if 'reset_parameters' in dir(module):
                if callable(module.reset_parameters):
                    module.reset_parameters()
                    if verbose:
                        print("Reset parameters in {}".format(module))


In [9]:
"""
Particle approximations for posterior of Bayesian neural net used in SVGD.

References:
    Liu, Qiang, and Dilin Wang. "Stein variational gradient descent:
    A general purpose bayesian inference algorithm." NIPS. 2016.

methods:
    __init__
    forward
    compute_loss
    compute_mse_nlp
    predict
    propagate

Note: 
`torch.distributions` is not much used in this implementation to keep simple.
Also we trade computing for memory by using for-loop rather than in a batch way.
"""
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


class BayesNN(nn.Module):
    """Class for Bayesian NNs with Stein Variational Gradient Descent.
    Not for usage independently.
    
    Bayesian NNs: y = f(x, w) + n

    uncertain weights:
            w_i ~ Normal(w_i | mu=0, 1 / alpha) 
            alpha ~ Gamma(alpha | shape=1, rate=0.05) (shared)
            --> w_i ~ StudentT(w_i | mu=0, lambda=shape/rate, nu=2*shape)
            Parameterization of StudentT in Bishop p.103 Eq. (2.159)

    Assumptions on noise:
        Additive, Gaussian, homoscedastic (independent of input), 
        output wise (same for every pixels in the output).
            n ~ Normal(0, 1 / beta)
            beta ~ Gamma(beta | shape=2, rate=2e-6)

    Hyperparameters for weights and noise are pre-defined based on heuristic.

    Given a deterministic `model`, initialize `n_samples` replicates
    of the `model`. (plus `n_samples` of noise precision realizations)

    `model` must implement `reset_parameters` method for the replicates
    to have different initial realizations of uncertain parameters.

    References:
        Liu, Qiang, and Dilin Wang. "Stein variational gradient descent:
        A general purpose bayesian inference algorithm."
        Advances In Neural Information Processing Systems. 2016.

    Args:
        model (nn.Module): The deterministic NN to be instantiated `n_samples` 
            times
        data_loader (utils.data.DataLoader): For training and testing
        n_samples (int): Number of samples for uncertain parameters
    """
    def __init__(self, model, n_samples=20):
        super(BayesNN, self).__init__()
        if not isinstance(model, nn.Module):
            raise TypeError("model {} is not a Module subclass".format(
                torch.typename(model)))

        self.n_samples = n_samples

        # w_i ~ StudentT(w_i | mu=0, lambda=shape/rate, nu=2*shape)
        # for efficiency, represent StudentT params using Gamma params
        self.w_prior_shape = 1.
        self.w_prior_rate = 0.05
        
        # noise variance 1e-6: beta ~ Gamma(beta | shape, rate)
        self.beta_prior_shape = 2.
        self.beta_prior_rate = 1.e-6

        # replicate `n_samples` instances with the same network as `model`
        instances = []
        for i in range(n_samples):
            new_instance = copy.deepcopy(model)
            # initialize each model instance with their defualt initialization
            # instead of the prior
            new_instance.reset_parameters()
            print('Reset parameters in model instance {}'.format(i))
            instances.append(new_instance)
        self.nnets = nn.ModuleList(instances)
        del instances

        # log precision (Gamma) of Gaussian noise
        log_beta = Gamma(self.beta_prior_shape, 
                         self.beta_prior_rate).sample((self.n_samples,)).log()
        for i in range(n_samples):
            self.nnets[i].log_beta = Parameter(log_beta[i])

        print('Total number of parameters: {}'.format(self._num_parameters()))

    def _num_parameters(self):
        count = 0
        for name, param in self.named_parameters():
            # print(name)
            count += param.numel()
        return count

    def __getitem__(self, idx):
        return self.nnets[idx]

    @property
    def log_beta(self):
        return torch.tensor([self.nnets[i].log_beta.item() 
            for i in range(self.n_samples)], device=device)

    def forward(self, input):
        """Computes all the `n_samples` NN output
        Args:
            input: N x iC x iH x iW

        Return:
            output: S x N x oC x oH x oW
        """
        output = []
        for i in range(self.n_samples):
            output.append(self.nnets[i].forward(input))
        output = torch.stack(output)

        return output

    def _log_joint(self, index, output, target, ntrain):
        """Log joint probability or unnormalized posterior for single model
        instance. Ignoring constant terms for efficiency.
        Can be implemented in batch computation, but memory is the bottleneck.
        Thus here we trade computation for memory, e.g. using for loop.

        Args:
            index (int): model index, 0, 1, ..., `n_samples`
            output (Tensor): B x oC x oD x oH x oW
            target (Tensor): B x oC x oD x oH x oW
            ntrain (int): total number of training data, mini-batch is used to
                evaluate the log joint prob

        Returns:
            Log joint probability (zero-dim tensor)
        """
        # Normal(target | output, 1 / beta * I)
        log_likelihood = ntrain / output.size(0) * (
                            - 0.5 * self.nnets[index].log_beta.exp()
                            * (target - output).pow(2).sum()
                            + 0.5 * target.numel() * self.nnets[index].log_beta)
        # log prob of prior of weights, i.e. log prob of studentT
        log_prob_prior_w = torch.tensor(0.).to(device)
        for param in self.nnets[index].features.parameters():
            log_prob_prior_w += \
                torch.log1p(0.5 / self.w_prior_rate * param.pow(2)).sum()
        log_prob_prior_w *= -(self.w_prior_shape + 0.5)
        # log prob of prior of log noise-precision (NOT noise precision)
        log_prob_prior_log_beta = ((self.beta_prior_shape-1.0) * self.nnets[index].log_beta \
                    - self.nnets[index].log_beta.exp() * self.beta_prior_rate)
        return log_likelihood + log_prob_prior_w + log_prob_prior_log_beta


    def _compute_mse_nlp(self, input, target, size_average=True, out=False):
        """Evaluate the MSE and Negative Log Probability.

        Args:
            input (Tensor): (N, iC, iH, iW)
            target (Tensor): (N, oC, oH, oW)
            size_average (bool)
            out (bool): If True, return output of `bayes_nn` w. `input`

        Returns:
            (mse, nlp) if `out` is False, o.w. (mse, nlp, output)
            where output is of size (S, N, oC, oH, oW)
        """
        # S x N x oC x oH x oW
        output = self.forward(input)
        # S x 1 x 1 x 1 x 1
        log_beta = self.log_beta.unsqueeze(-1).unsqueeze(-1).unsqueeze(-1).unsqueeze(-1).unsqueeze(-1)
        log_2pi_S = torch.tensor(0.5 * target[0].numel() * math.log(2 * math.pi)
                       + math.log(self.n_samples), device=device)
        # S x N
        # print(log_beta.exp().shape)
        # print(((target - output) ** 2).shape)
        exponent = - 0.5 * (log_beta.exp() * ((target - output) ** 2)).view(
            self.n_samples, target.size(0), -1).sum(-1) \
                    + 0.5 * target[0].numel() * self.log_beta.unsqueeze(-1)

        # n = target[0].numel()
        nlp = - log_sum_exp(exponent, dim=0).mean() + log_2pi_S
        mse = ((target - output.mean(0)) ** 2).mean()

        if not size_average:
            mse *= target.numel()
            nlp *= target.size(0)
        if not out:
            return mse, nlp
        else:
            return mse, nlp, output


    def predict(self, x_test):
        """
        Predictive mean and variance at x_test. (only average over w and beta)
        Args:
            x_test (Tensor): [N, *], test input
        """
        # S x N x oC x oH x oW
        y = self.forward(x_test)
        y_pred_mean = y.mean(0)
        # compute predictive variance per pixel
        # N x oC x oH x oW
        EyyT = (y ** 2).mean(0)
        EyEyT = y_pred_mean ** 2
        beta_inv = (- self.log_beta).exp()
        y_pred_var = beta_inv.mean() + EyyT - EyEyT

        return y_pred_mean, y_pred_var


    def propagate(self, mc_loader):
        """
        Mean and Variance statistics of predictive output distribution
        averaging over the input distribution, i.e. uncertainty propagation.

        First compute the conditional predictive mean and var given realizations
        of uncertain surrogate; then compute the statistics of those conditional
        statistics.

        Args:
            mc_loader (torch.utils.data.DataLoader): dataloader for the Monte 
                Carlo data (10,000 is used in this work)

            S: num of samples
            M: num of data
            D: output dimensions
        """
        # First compute conditional statistics
        # S x N x oC x oH x oW
        # self.cpu()
        # x_test = x_test.cpu()
        # print('here')

        # S x oC x oH x oW
        output_size = mc_loader.dataset[0][1].size()
        cond_Ey = torch.zeros(self.n_samples, *output_size, device=device)
        cond_Eyy = torch.zeros_like(cond_Ey)

        for _, (x_mc, _) in enumerate(mc_loader):
            x_mc = x_mc.to(device)
            # S x B x oC x oH x oW            
            y = self.forward(x_mc)
            cond_Ey += y.mean(1)
            cond_Eyy += y.pow(2).mean(1)
        cond_Ey /= len(mc_loader)
        cond_Eyy /= len(mc_loader)
        beta_inv = (- self.log_beta).exp()
        print('Noise variances: {}'.format(beta_inv))
        
        y_cond_pred_var = cond_Eyy - cond_Ey ** 2 \
                     + beta_inv.unsqueeze(-1).unsqueeze(-1).unsqueeze(-1)

        # compute statistics of conditional statistics
        return cond_Ey.mean(0), cond_Ey.var(0), \
               y_cond_pred_var.mean(0), y_cond_pred_var.var(0)



In [10]:
def reparameterization(mu, logvar):
    std = torch.exp(logvar / 2)
    eps = torch.randn_like(std)
    #randn_like: Returns a tensor with the same size as input that is filled with random numbers from a normal distribution with mean 0 and variance 1.
    #return: random gaussian sample from distribution with mu and exp(logvar/2)
    return mu + eps*std


class DenseResidualBlock(nn.Module):
    """
    The core module of paper: (Residual Dense Network for Image Super-Resolution, CVPR 18)
    """

    def __init__(self, filters, res_scale=0.2):
        super(DenseResidualBlock, self).__init__()
        self.res_scale = res_scale

        def block(in_features, non_linearity=True):
            layers = [nn.BatchNorm3d(in_features)]
            layers += [nn.ReLU(inplace=True)]
            layers += [nn.Conv3d(in_features, filters, 3, 1, 1, bias=True)] # does not change state size
            return nn.Sequential(*layers)

        self.b1 = block(in_features=1 * filters)
        self.b2 = block(in_features=2 * filters)
        self.b3 = block(in_features=3 * filters)
        self.b4 = block(in_features=4 * filters)
        self.b5 = block(in_features=5 * filters, non_linearity=False)
        self.blocks = [self.b1, self.b2, self.b3, self.b4, self.b5]

    def forward(self, x):
        inputs = x
        for block in self.blocks:
            out = block(inputs)
            inputs = torch.cat([inputs, out], 1)
        return out.mul(self.res_scale) + x


class ResidualInResidualDenseBlock(nn.Module):
    def __init__(self, filters, res_scale=0.2):
        super(ResidualInResidualDenseBlock, self).__init__()
        self.res_scale = res_scale
        self.dense_blocks = nn.Sequential(
            DenseResidualBlock(filters), DenseResidualBlock(filters), DenseResidualBlock(filters)#, DenseResidualBlock(filters)
        )

    def forward(self, x):
        return self.dense_blocks(x).mul(self.res_scale) + x
class Decoder(nn.Module):
    def __init__(self, inchannels=2, outchannels=1, filters=48, num_res_blocks=1,num_upsample=2):
        super(Decoder, self).__init__()

        # First layer. input size, inchannels x 2 x 8 x 16
        self.conv1 = nn.Conv3d(inchannels, filters, kernel_size=3, stride=1, padding=1)

        # state size. filters x 2 x 8 x 16
        # Residual blocks
        self.res_block1 = nn.Sequential(*[ResidualInResidualDenseBlock(filters) for _ in range(num_res_blocks+1)])
        self.transup1 = nn.Sequential(
            nn.BatchNorm3d(filters),
            nn.ReLU(inplace=True),
            nn.Upsample(size=(4, 21, 41), mode='nearest'),
            nn.Conv3d(filters, filters, kernel_size=3, stride=1, padding=1), #does not change state size
        )
        self.res_block2 = nn.Sequential(*[ResidualInResidualDenseBlock(filters) for _ in range(num_res_blocks)])
        self.transup2 = nn.Sequential(
            nn.BatchNorm3d(filters),
            nn.ReLU(inplace=True),
            nn.Upsample(size=(8, 41, 81), mode='nearest'),
            nn.Conv3d(filters, outchannels, kernel_size=3, stride=1, padding=(0,1,1)), # reduce the first dimension by 2
        )


    def forward(self, z):
        # x: in_channels x 2 x 8 x 16
        out1 = self.conv1(z)          # filters x 2 x 8 x 16
        out2 = self.res_block1(out1)   # filters x 2 x 8 x 16
        out = torch.add(out1, out2)   # filters x 2 x 8 x 16
        out3 = self.transup1(out)      # filters x 4 x 16 x 32
        out4 = self.res_block2(out3)   # filters x 4 x 16 x 32

        img = self.transup2(out4)     # outchannels x 6 x 32 x 64

        return img

    def _n_parameters(self):
        n_params= 0
        for name, param in self.named_parameters():
            n_params += param.numel()
        return n_params

# load CAAE and Bayesian DenseED models

In [11]:
'''Load the CAAE model first'''
cuda = True if torch.cuda.is_available() else False
n_train = 23000
n_test = 100
batch_size = 64
n_epochs = 50
lr = 0.0002 ## adam learning rate
lw = 0.01 ## "adversarial loss weight"

current_dir = "/content/drive/MyDrive/react_inverse/CAAE/"
date = 'experiments/Feb_14_CAAE3D'
exp_dir = current_dir + date + "/N{}_Bts{}_Eps{}_lr{}_lw{}".\
    format(n_train, batch_size, n_epochs, lr, lw)

output_dir = exp_dir + "/predictions"
model_dir = exp_dir

nf, d, h, w = 2, 2, 11, 21

# Initialize decoder
decoder = Decoder(inchannels=nf)
decoder.load_state_dict(torch.load(model_dir + '/AAE_decoder_epoch{}.pth'.format(n_epochs)))
if cuda:
    decoder.cuda()

decoder.eval()

Decoder(
  (conv1): Conv3d(2, 48, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (res_block1): Sequential(
    (0): ResidualInResidualDenseBlock(
      (dense_blocks): Sequential(
        (0): DenseResidualBlock(
          (b1): Sequential(
            (0): BatchNorm3d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (1): ReLU(inplace=True)
            (2): Conv3d(48, 48, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
          )
          (b2): Sequential(
            (0): BatchNorm3d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (1): ReLU(inplace=True)
            (2): Conv3d(96, 48, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
          )
          (b3): Sequential(
            (0): BatchNorm3d(144, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (1): ReLU(inplace=True)
            (2): Conv3d(144, 48, kernel_size=(3, 3, 3), stride=(1, 1, 1), paddin

In [61]:
exp_dir = "/content/drive/MyDrive/react_inverse/Bayesian_DenseED"
all_over_again = 'Apr_04'
n_samples = 20
n_train = 1000
batch_size = 40
lr = 0.005
lr_noise = 0.01
n_epochs = 300
ckpt_epoch = None
run_dir = exp_dir + '/' + all_over_again\
    + '/ns{}_ntr{}_bt{}_lr{}_lrn{}_ep{}'.format(
        n_samples, n_train, batch_size, lr,
        lr_noise, n_epochs)
pred_dir = run_dir + '/predictions'
ckpt_dir = run_dir + '/checkpoints'
nic = 3
noc = 2
blocks = [3,6,3]
growth_rate = 40
init_features = 48
drop_rate = 0.
bn_size = 8
bottleneck = False

dense_ed = DenseED(
    in_channels=nic, 
    out_channels=noc, 
    blocks=blocks,
    growth_rate=growth_rate, 
    num_init_features=init_features,
    drop_rate=drop_rate,
    bn_size=bn_size,
    bottleneck=bottleneck,
)
# print(dense_ed)
# Bayesian NN
bayes_nn = BayesNN(dense_ed, n_samples=n_samples).to(device)
# load the pre-trained model
if ckpt_epoch is not None:
    checkpoint = ckpt_dir + '/model_epoch{}.pth'.format(ckpt_epoch)
else:
    checkpoint = ckpt_dir + '/model_epoch{}.pth'.format(n_epochs)
bayes_nn.load_state_dict(torch.load(checkpoint))
print('Loaded pre-trained model: {}'.format(checkpoint))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # training on GPU or CPU


Reset parameters in model instance 0
Reset parameters in model instance 1
Reset parameters in model instance 2
Reset parameters in model instance 3
Reset parameters in model instance 4
Reset parameters in model instance 5
Reset parameters in model instance 6
Reset parameters in model instance 7
Reset parameters in model instance 8
Reset parameters in model instance 9
Reset parameters in model instance 10
Reset parameters in model instance 11
Reset parameters in model instance 12
Reset parameters in model instance 13
Reset parameters in model instance 14
Reset parameters in model instance 15
Reset parameters in model instance 16
Reset parameters in model instance 17
Reset parameters in model instance 18
Reset parameters in model instance 19
Total number of parameters: 63578420
Loaded pre-trained model: /content/drive/MyDrive/react_inverse/Bayesian_DenseED/Apr_04/ns20_ntr1000_bt40_lr0.005_lrn0.01_ep300/checkpoints/model_epoch300.pth


# ILUES

In [13]:
class ILUES:

    def ilues_select(self,Par, x, y):
        self.Par = Par
        Cd = np.eye(Par.Nobs)
        for i in range(Par.Nobs):
            Cd[i, i] = Par.sd[i]**2 #covariance matrix for the error

        meanxf = np.tile(np.mean(x1, axis=1, keepdims=True), (1, Par.Ne))
        Cm = np.matmul((x1 - meanxf), (x1 - meanxf).T)/(Ne - 1) # auto-covariance of the prior parameters
        
        J1 = np.zeros((Par.Ne,1))
        for i in range(Par.Ne):
            J1[i,] = np.matmul((yf[:,i]-Par.obs).T/Cd, (yf[:,i]-Par.obs))[0,0]

        xa = np.zeros(x1.shape)  # define the updated ensemble   
        for j in range(Par.Ne):
            xa[:,j] = self.local_update(xf,yf,Cm,sd,obs,alpha,beta,J1,j)
        return xa


    def local_update(self,xf,yf,Cm,sd,obs,alpha,beta,J1,jj):
        # The local updating scheme used in ILUES
        Ne = xf.shape[1]
        xr = xf[:, jj]
        xr = np.tile(np.reshape(xr, (-1, 1)), (1, Ne))
        J = np.matmul((xf-xr).T/Cm, (xf-xr))
        J2 = np.diag(J)
        J3 = J1/np.max(J1) + J2/np.max(J2)
        M = np.ceil(Ne*alpha)

        J3min, index = np.min(J3), np.unravel_index(np.argmin(J3, axis=None), J3.shape)
        xl = xf[:, index]
        yl = yf[:, index]
        alpha_ = J3min / J3
        alpha_[index] = 0
        index1 = self.RouletteWheelSelection(alpha_, M-1)
        xl1 = xf[:, index1]
        yl1 = yf[:, index1]
        xl = np.asarray([xl, xl1])
        yl = np.asarray([yl, yl1])
        xu =  self.update_para(xl,yl,self.Par.para_range,sd*beta,obs)
        a = time.time()
        np.random.seed(int(a))
        xest = xu[:,np.random.permutation(M)]
        return xest

    def RouletteWheelSelection(self,V,m):
        '''
        Input:
              V           -----fitness criterion
              m           -----number of individuals to be chosen
        Output:
              index       -----index of the chosen individuals
        '''
        n = V.shape[1]
        if np.max(V)==0 and np.min(V)==0:
            index=np.ceil(np.random.uniform(size=(1,m))*n)
        else:
            temindex= np.nonzero(V)
            n=len(temindex)
            V=V[temindex]

            V=np.cumsum(V)/np.sum(V)

            pp=np.random.uniform(size=(1,m))
            index = []
            for i in range(m):

                while True:
                    flag = True
                    for j in range(n):
                        if pp[i] < V[j]:
                            index.append(j)
                            V[j] = 0
                            flag = False
                            break

                    if flag:
                        pp[i] = np.random.uniform()
                    else:
                        break
        return np.array(index)


    def update_para(self,xf,yf,para_range, sd,obs):
        #Update the model parameters via the ensemble smoother
        para_range = se
        Npar = xf.shape[0]
        Ne = xf.shape[1]
        Nobs = len(obs)

        Cd = np.eye(Nobs)
        for i in range(Nobs):
            Cd[i,i] = sd(i)**2
        meanxf = np.tile(np.mean(xf, axis=1, keepdims=True), (1, Ne))
        meanyf = np.tile(np.mean(yf, axis=1, keepdims=True), (1, Ne))	
        Cxy = np.matmul((xf - meanxf), (yf - meanyf).T)/(Ne - 1)
        Cyy = np.matmul((yf - meanyf), (yf - meanyf).T)/(Ne - 1)

        kgain = linalg.lstsq((Cyy + Cd).T, Cxy.T)[0].T ##Cxy/(...), A/B = (B'\A')', b/a: linalg.lstsq(a.T, b.T)
        obse = np.tile(np.reshape(obs,(-1,1)),(1,Ne)) +\
          np.random.normal(np.zeros((Nobs,Ne)),np.tile(np.reshape(sd, (-1, 1)),(1,Ne)))
        xa = xf + np.matmul(kgain, (obse - yf))

        ##when the updated parameters exceed the range	
        for i in range(Ne):
            for j in range(Npar):
                if xa[j,i] > para_range[1, j]:
                    xa[j,i] = (para_range[1, j] + xf[j,i])/2
                elif xa[j,i] < para_range[0, j]:
                    xa[j,i] = (para_range[0, j] + xf[j,i])/2
        
        return xa

In [14]:
def plot_3d(data, title='', cut=None):
    data = np.transpose(data, (2, 1, 0))
    data = np.flip(data, axis=2)
    filled = np.ones(data.shape)
    if cut is not None:
        filled[cut[2]:, :cut[1], (6-cut[0]):] = 0
    x, y, z = np.indices(np.array(filled.shape) + 1)
    
    v1 = np.linspace(np.min(data),np.max(data), 8, endpoint=True)
    norm = matplotlib.colors.Normalize(vmin=np.min(data), vmax=np.max(data))
    
    fig = plt.figure()
    ax = plt.axes(projection='3d')
    ax.voxels(x, y, z, filled, facecolors=plt.cm.jet(norm(data)), edgecolors=None, 
              )
    ax.set_box_aspect((250, 125, 50))
    
    m = cm.ScalarMappable(cmap=plt.cm.jet, norm=norm)
    m.set_array([])
    fig.colorbar(m, ax=ax, fraction=0.015, pad=0.04,ticks=v1,)
    ax.set_axis_off()
    plt.tight_layout()
    # ax.set_title(title)
    # plt.savefig(title+'.pdf',bbox_inches='tight')
    return fig
# fig = plot_3d(log_K[0,], title='', cut=[3, 13+1, 20-1])

# ILUES support functions

In [64]:
def to_numpy(input):
    if isinstance(input, torch.Tensor):
        return input.cpu().detach().numpy()
    elif isinstance(input, np.ndarray):
        return input
    else:
        raise TypeError('Unknown type of input, expected torch.Tensor or '\
            'np.ndarray, but got {}'.format(type(input)))

def run_surrogate(X, Par, model):
    ya = np.zeros((Par.Ne, Par.Nobs))
    model.eval()
    start = time.time()
    log_K, source = gen_input4net(X, Par)
    end1 = time.time()
    print('Generating logK and source took ', end1-start, ' s.')
    for ind in range(Par.Ne):
        x = np.zeros((1,3,6,41,81))           # three input channels: hydraulic conductivity field, source term, previous concentration field
        y = np.zeros( (Par.Nt,2,6,41,81)) # two output channles: concentration and head fields
        y_i_1 = np.zeros((6,41,81))   # y_0 = 0

        for i in range(Par.Nt):
            x[0,0,:,:,:] = y_i_1          # the i-1)^th predicted concentration field, which is treated as an input channel
            x[0,1,:,:,:] = log_K[ind]           # hydraulic conductivity
            x[0,2,:,:,:] = source[ind][i]      # source rate
            x_tensor = Tensor(x)
            with torch.no_grad():
                y_hat = model.forward(x_tensor).mean(0)
            to_numpy(x_tensor)
            y_hat = y_hat.data.cpu().numpy()
            y[i] = y_hat
            y_i_1 = y_hat[0,0,:,:,:]      # the updated (i-1)^th predicted concentration field

        y_pred = np.full( (Par.Nt + 1,6,41,81), 0.0)
        y_pred[:Par.Nt] = y[:,0]   # the concentration fields at Nt time instances
        y_pred[Par.Nt]  = y[0,1]   # the hydraulic head field
        ya[ind, :] = np.reshape(get_obs(sensor, y_pred),(-1,))  # get the simulated outputs at observation locations using interpolation
    end2 = time.time()
    print('Running {} samples took '.format(Par.Ne), end2-end1, ' s.')
    return ya
   
def gen_input4net(X, Par, batch_size=100):
    '''generate batch input'''
    ## log conductivity field
    log_K = np.zeros((Par.Ne, 6, 41, 81))
    latent_z = X[:Par.Nlat, :] ## 924, Ne
    assert Par.Ne % batch_size ==0, 'Number of samples cannot be divided by the batch size.'
    for i in range(int(Par.Ne/batch_size)):
        batch_z = latent_z[:, i*batch_size:(i+1)*batch_size].copy()
        batch_z = torch.reshape(
          Tensor(batch_z), (-1, nf, d, h, w)
          )
        log_K[i*batch_size:(i+1)*batch_size,] = to_numpy(decoder(batch_z)).reshape(-1, 6, 41, 81)
        to_numpy(batch_z)
    ## source loc
    y_wel_samp = X[Par.Nlat, :]
    x_wel_samp = X[Par.Nlat+1, :]
    Sy_id, Sx_id = step_loc(y_wel_samp, x_wel_samp)
    ## source rate
    source_rate = X[Par.Nlat+2:, :] #shape: 5,Ne
    source = np.zeros((Par.Ne, Par.Nt, 6, 41, 81,))
    for j in range(Par.Nt_re): #j'th timestep of release
        for i in range(Par.Ne): #i'th sample
            source[i, j, 3, Sy_id[i], Sx_id[i]] = source_rate[j,i]

    return log_K, source

def get_obs(sensor, y_pred):

    y_sim_obs = []
    for y in y_pred:
        y_sim_obs.append(y[Par.sensor])

    return y_sim_obs

def gen_init(Par, seed=888):
    x = np.zeros((Par.Npar, Par.Ne))
    np.random.seed(seed)
    ## log_K
    x[:Par.Nlat, :] = np.random.randn(Par.Nlat, Par.Ne)
    ## release locations
    y_wel = np.array([125, 125*3, 125*5, 125*7, 125*9])
    x_wel = np.array([125, 125*3, 125*5, 125*7])
    # wells = {i: [y_wel[i], x_wel[i]] for i in range(len(y_wel))}
    y_wel_samp = np.random.choice(y_wel, Par.Ne)
    x_wel_samp = np.random.choice(x_wel, Par.Ne)

    x[Par.Nlat,:] = y_wel_samp
    x[Par.Nlat+1,:] = x_wel_samp
    ## release concentration for 5 periods
    q = np.random.uniform(low=100, high=1000, size=(5, Par.Ne)).astype(int)
    x[Par.Nlat+2:,:] = q
    return x

'''MAKE A FUNCTION TO STEPWISE THE WELL LOCATIONS'''
def step_loc(y_loc, x_loc):
    '''
    x_loc: (Ne, ), convert from meters to index,
    y_loc: (Ne, ), convert from meters to index.
    '''
    dy = 1250/40
    dx = 2500/80
    y_wel = np.array([125, 125*3, 125*5, 125*7, 125*9])
    x_wel = np.array([125, 125*3, 125*5, 125*7])
    N = len(x_loc)
    x_dist_wel = np.array(
        [
            [np.abs(x_loc[i] - x_wel[j]) for j in range(len(x_wel))] 
            for i in range(N)
        ]
    )
    y_dist_wel = np.array(
        [
            [np.abs(y_loc[i] - y_wel[j]) for j in range(len(y_wel))] 
            for i in range(N)
        ]
    )
    y_wel = (y_wel)//dy
    x_wel = (x_wel)//dx
    y_loc_ind = np.array([y_wel[np.argmin(y_dist_wel, axis=1)[i]] for i in range(N)])
    x_loc_ind = np.array([x_wel[np.argmin(x_dist_wel, axis=1)[i]] for i in range(N)])
    
    return y_loc_ind.astype(int), x_loc_ind.astype(int)

Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

In [16]:
## load the measurements, meas[:,0]: the measurements, meas[:, 1]: sigma for meas error.
with open('/content/drive/MyDrive/react_inverse/dense_ed_3d/sensor_loc.pkl', 'rb') as file:
    sensor = pk.load(file)
with open('/content/drive/MyDrive/react_inverse/ILUES/output.pk', 'rb') as file:
    [conc, heads] = pk.load(file)
conc, heads = np.array(conc), np.array(heads)
conc[np.where(conc<0)] = 0
meas_sig = [0.05*conc[i][sensor] for i in range(len(conc))] + [0.05*heads[sensor]]
meas_orig = [conc[i][sensor] for i in range(len(conc))] + [heads[sensor]]
np.random.seed(888)
meas_data = [meas_orig[i] + np.random.normal(0, meas_sig[i]) for i in range(len(meas_orig))]
meas = np.array(
    [np.reshape(meas_data, (-1,)), np.reshape(meas_sig, (-1))]
).T

In [ ]:
# for j in range(len(meas_data[0])):
#     plt.plot([meas_data[i][j] for i in range(len(meas_data))])
def vis_sensors(meas,sensor, t):
    fig = plt.figure(figsize=(20, 20))
    fig.suptitle(t)
    for i in range(len(meas[0])):
        sen = [None]*len(meas)
        for j in range(len(meas)-1):
            sen[j] = meas[j][i]
        ax = fig.add_subplot(12,10,i+1)
        ax.plot(sen)
        ax.set_title(str(sensor[0][i])+','+str(sensor[1][i])+','+str(sensor[2][i]))
    
    fig.tight_layout()
    fig.subplots_adjust(top=0.88)
    name = t + '.pdf'
    fig.savefig(name, format='pdf')
    plt.show()

vis_sensors(meas_data, np.where(sensor), 'conc_sensor')

In [38]:
np.array(meas_data).shape

(11, 120)

In [56]:
# with open('obs_sd.pkl', 'rb') as file:
#     meas = pkl.load(file)
Par=AttrDict()
Par.obs = meas[:,0] # observations
Par.sd  = meas[:,1] # standard deviations of the observation error
Par.Nobs = meas.shape[0]
Par.sensor = sensor
Par.Nlat = nf*d*h*w
Par.Nt = 10
Par.Nt_re = 5
Par.Npar = Par.Nlat + 2 + Par.Nt_re 

Par.N_iter = 20
Par.alpha = 0.1  # a scalar within [0 1]
Par.Ne = 600
Par.beta = np.sqrt(Par.N_iter)
Par.para_range = np.asarray(
    [
        [0, 0] + [100 for i in range(Par.Nt_re)] + [-5 for i in range(Par.Nlat)],
        [1200, 812.5] + [1000 for i in range(Par.Nt_re)] + [5 for i in range(Par.Nlat)]
    ]
)
X = gen_init(Par)
print('X shape: ', X.shape)

x shape:  (931, 600)


In [65]:
ya = run_surrogate(X, Par, bayes_nn)

Generating logK and source took  5.382914304733276  s.
Running 600 samples took  1030.2767112255096  s.
